In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import tensorflow as tf
from conversion_tf import GEMMDecisionTreeImpl, TreeTraversalDecisionTreeImpl, PerfectTreeTraversalDecisionTreeImpl
import numpy as np
from hummingbird.ml import convert
import torch

2023-02-07 00:33:24.076309: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# __Important__ <br>
__DONT__ run all off the examples, because this machine will run out of memory <br>
__ALSO__ the __Ptt__ example uses a small RF because of memory <br>
__GENERALLY__ all 3 methods work correctly and are tflite convertable <br>
__BUT__ they arent __Edge TPU__ compatible yet

In [2]:
def representative_dataset():
    for _ in range(100):
      data = np.random.uniform(low=0., high=8., size=(1,8))
      yield [data.astype(np.float32)]
 
tf.config.run_functions_eagerly(True)

forest = RandomForestClassifier(n_estimators=105)
forest2 = RandomForestClassifier(n_estimators=25)
X, y = make_classification(n_samples=1300, n_features=8,
                           n_informative=4, n_redundant=1,
                           random_state=0, shuffle=True,
                           n_classes=4)

x_train, y_train = X[:1000], y[:1000]
x_test, y_test = X[1000:], y[1000:]

forest.fit(x_train, y_train)
forest2.fit(x_train, y_train)

X = tf.constant([1, 2, 3, 4, 5, 6, 7, 8], shape=[1, 8], dtype=tf.int8)
X_float = tf.constant([1., 2., 3., 4., 5., 6., 7., 8.], shape=[1, 8])

2023-02-07 00:33:34.072296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 00:33:34.073122: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


## TTDecisionTreeTraversalImpl

In [ ]:
conv_model = convert(forest, 'torch', extra_config={"tree_implementation":"tree_trav"})

y_mod_pred_tt, y_mod_tt = conv_model.model._operators[0].forward((torch.tensor([[1., 2., 3., 4., 5., 6., 7., 8.]])))

model_tt = TreeTraversalDecisionTreeImpl(forest)

y_pred_tt, y_tt = model_tt(X_float)

concrete_func = model_tt.__call__.get_concrete_function()

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], model_tt)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  
converter.inference_output_type = tf.int8  
tflite_model_tt = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite_model_tt)
interpreter.allocate_tensors()

output = interpreter.get_output_details()[0]  
input = interpreter.get_input_details()[0]  

interpreter.set_tensor(input['index'], X)
interpreter.invoke()
y_lite_tt = interpreter.get_tensor(output['index'])

y_pred_lite_tt = np.argmax(y_lite_tt, axis=1)

In [ ]:
print(f'############################################ TT ###############################################################\n')
print(f'Prediction original    : {y_mod_pred_tt.detach().numpy()[0]} \t Class Confidence: {y_mod_tt.detach().numpy()[0]}')
print(f'Prediction tf module   : {y_pred_tt.numpy()[0]} \t Class Confidence: {y_tt.numpy()[0]}')
print(f'Prediction tflite quant: {y_pred_lite_tt[0]} \t Class Confidence: {y_lite_tt[0]}')
print(f'\n###############################################################################################################\n')

## PerfectDecisionTreeTraversalImpl

In [3]:
conv_model = convert(forest2, 'torch', extra_config={"tree_implementation":"perf_tree_trav"})

y_mod_pred_ptt, y_mod_ptt = conv_model.model._operators[0].forward((torch.tensor([[1., 2., 3., 4., 5., 6., 7., 8.]])))

model_ptt = PerfectTreeTraversalDecisionTreeImpl(forest2)

y_pred_ptt, y_ptt = model_ptt(X_float)

concrete_func = model_ptt.__call__.get_concrete_function()

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], model_ptt)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  
converter.inference_output_type = tf.int8  
tflite_model_ptt = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite_model_ptt)
interpreter.allocate_tensors()

output = interpreter.get_output_details()[0]  
input = interpreter.get_input_details()[0]  

interpreter.set_tensor(input['index'], X)
interpreter.invoke()
y_lite_ptt = interpreter.get_tensor(output['index'])

y_pred_lite_ptt = np.argmax(y_lite_ptt, axis=1)

INFO:tensorflow:Assets written to: /tmp/tmphad3nn23/assets


2023-02-07 00:34:03.608154: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-02-07 00:34:03.608175: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-02-07 00:34:03.608636: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmphad3nn23
2023-02-07 00:34:03.915249: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-02-07 00:34:03.915284: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmphad3nn23
2023-02-07 00:34:03.915953: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-02-07 00:34:04.947629: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2023-02-07 00:34:04.966687: I tensorflow/cc/save

Estimated count of arithmetic ops: 925  ops, equivalently 462  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8
2023-02-07 00:34:24.052463: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1989] Estimated count of arithmetic ops: 925  ops, equivalently 462  MACs



Estimated count of arithmetic ops: 925  ops, equivalently 462  MACs


In [7]:
print(f'\n############################################ PTT ##############################################################\n')
print(f'Prediction original    : {y_mod_pred_ptt.detach().numpy()[0]} \t Class Confidence: {y_mod_ptt.detach().numpy()[0]}')
print(f'Prediction tf module   : {y_pred_ptt.numpy()[0]} \t Class Confidence: {y_ptt.numpy()[0]}')
print(f'Prediction tflite quant: {y_pred_lite_ptt[0]} \t Class Confidence: {y_lite_ptt[0]}')
print(f'\n###############################################################################################################\n')


############################################ PTT ##############################################################

Prediction original    : 1 	 Class Confidence: [0.19999999 0.43999997 0.         0.35999995]
Prediction tf module   : 1 	 Class Confidence: [0.19999999 0.43999997 0.         0.36      ]
Prediction tflite quant: 0 	 Class Confidence: [ 124  118 -128  117]

###############################################################################################################



## GEMMDecisionTreeImpl

In [ ]:
model_gemm = GEMMDecisionTreeImpl(forest)

conv_model = convert(forest, 'torch', extra_config={"tree_implementation":"gemm"})

y_mod_pred_gemm, y_mod_gemm = conv_model.model._operators[0].forward((torch.tensor([[1., 2., 3., 4., 5., 6., 7., 8.]])))

y_pred_gemm, y_gemm = model_gemm(X_float)

concrete_func = model_gemm.__call__.get_concrete_function()

converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func], model_gemm)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  
converter.inference_output_type = tf.int8  
tflite_model_gemm = converter.convert()

interpreter = tf.lite.Interpreter(model_content=tflite_model_gemm)
interpreter.allocate_tensors()

output = interpreter.get_output_details()[0]  
input = interpreter.get_input_details()[0]  


interpreter.set_tensor(input['index'], X)
interpreter.invoke()
y_lite_gemm = interpreter.get_tensor(output['index'])

y_pred_lite_gemm = np.argmax(y_lite_gemm, axis=1)


In [ ]:
print(f'\n############################################ GEMM #############################################################\n')
print(f'Prediction original    : {y_mod_pred_gemm.detach().numpy()[0]} \t Class Confidence: {y_mod_gemm.detach().numpy()[0]}')
print(f'Prediction tf module   : {y_pred_gemm.numpy()[0]} \t Class Confidence: {y_gemm.numpy()[0]}')
print(f'Prediction tflite quant: {y_pred_lite_gemm[0]} \t Class Confidence: {y_lite_gemm[0]}')
print(f'\n###############################################################################################################\n')